In [ ]:
import discord
from discord.ext import commands
import os
from dotenv import load_dotenv
import joblib

# Load environment variables
load_dotenv()

# Set up the bot
intents = discord.Intents.default()
intents.message_content = True
bot = commands.Bot(command_prefix='!', intents=intents)

model = None

def load_model():
    global model
    MODEL_PATH = 'model.pkl'  
    try:
        model = joblib.load(MODEL_PATH)
        print("Model loaded successfully.")
    except FileNotFoundError:
        print(f"Error: Model file not found at {MODEL_PATH}")
        return False
    except Exception as e:
        print(f"Error loading the model: {e}")
        return False
    return True

def suggest_project(input_topic):
    if model is None:
        return "Sorry, the project suggestion model is not available."
    try:
        processed_input = input_topic.lower()
        prediction = model.predict([processed_input])
        return f"The suggested project category is: {prediction[0]}"
    except Exception as e:
        print(f"Error in suggest_project: {e}")
        return "Sorry, I encountered an error while processing your request."

@bot.command(name='suggest')
async def suggest(ctx, *, input_topic: str):
    suggestion = suggest_project(input_topic)
    await ctx.send(suggestion)

@bot.command(name='commands')
async def custom_help(ctx):
    commands_list = [
        "!suggest [topic] - Get a project suggestion based on a topic",
        "!commands - Show this help message"
    ]
    help_text = "Available commands:\n" + "\n".join(commands_list)
    await ctx.send(help_text)

@bot.event
async def on_command_error(ctx, error):
    if isinstance(error, commands.CommandNotFound):
        await ctx.send("Sorry, I don't recognize that command. ")
    elif isinstance(error, commands.MissingRequiredArgument):
        await ctx.send("Please provide a topic after the !suggest command.")
    else:
        await ctx.send(f"An error occurred: {str(error)}")

@bot.event
async def on_ready():
    print(f'{bot.user} has connected to Discord!')
    print(f'Bot is active in {len(bot.guilds)} servers.')

if _name_ == "_main_":
    token = os.getenv('DISCORD_TOKEN')
    if not token:
        print("Error: DISCORD_TOKEN not found in .env file")
        exit(1)

    if load_model():
        bot.run(token)
    else:
        print("Failed to load the model.")

Just upload this..